In [64]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

**Try fitting `XGBClassifier` with minimal data**

In [71]:
X = np.expand_dims(np.array([1]), 1)
Y = np.array([1])

In [72]:
classifier = XGBClassifier(
    n_estimators=2000,
    max_depth=4,
    min_child_weight=2,
    gamma=0.9,
    subsample=0.8,
    colsample_bytree=.8,
    nthread=-1,
    # objective='multi:softprob',
    scale_pos_weight=1)
classifier.fit(X, Y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.9, learning_rate=0.1,
       max_delta_step=0, max_depth=4, min_child_weight=2, missing=None,
       n_estimators=2000, n_jobs=1, nthread=-1,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.8)

**Can error from the previous notebook be reproduced?**

In [27]:
X_2 = np.expand_dims(np.array([1]*12096), 1)
Y_2 = np.array([1]*12096)

In [28]:
print(X_2.shape, Y_2.shape)

(12096, 1) (12096,)


In [29]:
classifier = XGBClassifier(
    n_estimators=2000,
    max_depth=4,
    min_child_weight=2,
    gamma=0.9,
    subsample=0.8,
    colsample_bytree=.8,
    nthread=-1,
    # objective='multi:softprob',
    scale_pos_weight=1)
classifier.fit(X_2, Y_2)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.9, learning_rate=0.1,
       max_delta_step=0, max_depth=4, min_child_weight=2, missing=None,
       n_estimators=2000, n_jobs=1, nthread=-1,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=0.8)

Despite the same dimensions it seems to work

**Diversify labels**

In [59]:
MIN_LABEL=1
MAX_LABEL=7

def get_predictions():
    return np.random.randint(MIN_LABEL, MAX_LABEL+1, 12096)

# Remember the inputs of XGBoost are predictions
X_3 = np.expand_dims(get_predictions(), axis=1)
Y_3 = get_predictions()

In [58]:
pd.DataFrame(X_3).describe()

,0
count,12096.000000
mean,3.964864
std,1.994952
min,1.000000
25%,2.000000
50%,4.000000
75%,6.000000
max,7.000000


In [49]:
pd.DataFrame(Y_3).describe()

,0
count,12096.000000
mean,3.993386
std,1.997962
min,1.000000
25%,2.000000
50%,4.000000
75%,6.000000
max,7.000000


In [55]:
print(X_3.shape, Y_3.shape)

(12096, 1) (12096,)


In [57]:
classifier = XGBClassifier(
    n_estimators=2000,
    max_depth=4,
    min_child_weight=2,
    gamma=0.9,
    subsample=0.8,
    colsample_bytree=.8,
    nthread=-1,
    # objective='multi:softprob',
    scale_pos_weight=1)
classifier.fit(X_3, Y_3)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.9, learning_rate=0.1,
       max_delta_step=0, max_depth=4, min_child_weight=2, missing=None,
       n_estimators=2000, n_jobs=1, nthread=-1, objective='multi:softprob',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=0.8)

**Can't seem to reproduce; Try reusing previous code**

In [61]:
df = pd.DataFrame.from_csv('train.csv')

In [78]:
REDUNDANT_COLS = ['Soil_Type7', 'Soil_Type15']
Y_COLUMN_NAME = 'Cover_Type'
X_4 = df.drop(REDUNDANT_COLS+[Y_COLUMN_NAME], axis=1)
Y_4 = df[[Y_COLUMN_NAME]]

In [79]:
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(X_4, Y_4[Y_COLUMN_NAME])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=7, p=2,
           weights='uniform')

In [80]:
ensemble_X = np.concatenate([knn.predict(X_4)])

In [81]:
ensemble_X = np.expand_dims(ensemble_X, axis=0)
ensemble_X

array([[5, 5, 2, ..., 6, 5, 5]])

In [83]:
ensemble_X.shape, Y_4[Y_COLUMN_NAME].shape

((1, 12096), (12096,))

In [86]:
classifier = XGBClassifier(
    n_estimators=2000,
    max_depth=4,
    min_child_weight=2,
    gamma=0.9,
    subsample=0.8,
    colsample_bytree=.8,
    nthread=-1,
    # objective='multi:softprob',
    scale_pos_weight=1)
classifier.fit(ensemble_X, Y_4)

/home/k8tems/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/k8tems/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBoostError: b'[22:52:41] src/objective/multiclass_obj.cc:43: Check failed: preds->Size() == (static_cast<size_t>(param_.num_class) * info.labels_.size()) SoftmaxMultiClassObj: label size and pred size does not match\n\nStack trace returned 10 entries:\n[bt] (0) /home/k8tems/anaconda3/lib/python3.6/site-packages/xgboost/./lib/libxgboost.so(dmlc::StackTrace[abi:cxx11]()+0x56) [0x7f68e4f411b6]\n[bt] (1) /home/k8tems/anaconda3/lib/python3.6/site-packages/xgboost/./lib/libxgboost.so(xgboost::obj::SoftmaxMultiClassObj::GetGradient(xgboost::HostDeviceVector<float>*, xgboost::MetaInfo const&, int, xgboost::HostDeviceVector<xgboost::detail::GradientPairInternal<float> >*)+0x34f) [0x7f68e4fdbdbf]\n[bt] (2) /home/k8tems/anaconda3/lib/python3.6/site-packages/xgboost/./lib/libxgboost.so(xgboost::LearnerImpl::UpdateOneIter(int, xgboost::DMatrix*)+0x311) [0x7f68e4f4d9a1]\n[bt] (3) /home/k8tems/anaconda3/lib/python3.6/site-packages/xgboost/./lib/libxgboost.so(XGBoosterUpdateOneIter+0x27) [0x7f68e50f

In [87]:
ensemble_X.shape

(1, 12096)

In [91]:
ensemble_X = np.expand_dims(ensemble_X.ravel(), axis=1)

In [92]:
classifier = XGBClassifier(
    n_estimators=2000,
    max_depth=4,
    min_child_weight=2,
    gamma=0.9,
    subsample=0.8,
    colsample_bytree=.8,
    nthread=-1,
    # objective='multi:softprob',
    scale_pos_weight=1)
classifier.fit(ensemble_X, Y_4)

/home/k8tems/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/k8tems/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.9, learning_rate=0.1,
       max_delta_step=0, max_depth=4, min_child_weight=2, missing=None,
       n_estimators=2000, n_jobs=1, nthread=-1, objective='multi:softprob',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=0.8)

### Conclusion

Just had to change the dimensions of inputs from (1, 12096) to (12096, 1)